# Automated UAS SkyWatch Standard Format Remark Functions & Script

## Origin Location: RWY
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 29, 2021

***
***How to Use:***

`This function will output the dataframe containing the complete UAS location information for Standard Format Remarks referencing a Runway (RWY).`
***

## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

# REMARK MUST REFERENCE A RUNWAY

### uas_rwy() 

This function determines the records for which the Remarks reference a Runway.

In [28]:
def uas_rwy(file):
    '''
    This function takes in a file (str) containing unidentified Remarks from SkyWatch and determines those that reference a 
    runway as the origin location for the UAS sighting. This function outputs a dataframe containing the Remark, the Runway 
    Location, the Reporting Facility (needed to find the airport in relation to the runway referenced), and the UAS Distance
    from the runway (if provided)
    
    @param file: a .csv file containing unidentified Remarks from SkyWaych
    '''
    
    # Read in .csv file provided by user
    unid_remarks = pd.read_csv(file)
    
    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = unid_remarks['REMARKS']

    # regular expression for any heading/direction
    rwy_regex = 'RWY\s?[0-9]*[A-Z]?'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        rwy = re.findall(rwy_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(rwy)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    remark_uas_loc_df = pd.DataFrame()
    remark_uas_loc_df['REMARKS'] = remark
    remark_uas_loc_df['UAS_Location_Runways'] = uas_loc
    
    # Get list of UAS locations from the above dataframe
    uas_loc = remark_uas_loc_df['UAS_Location_Runways'] 

    # If the regular expressions did not hit on any location information, pass it UNKN for now
    for i in range(len(remark_uas_loc_df)):
        if len(uas_loc[i]) == 0:
            uas_loc[i] = 'UNKN'
    
    # NOTE :: remark_uas_loc_df is the dataframe that contains ALL records
    #         rwy_df is the dataframe that contains ONLY RECORDS THAT REFERENCES RWY
    
    rwy_df = remark_uas_loc_df[remark_uas_loc_df['UAS_Location_Runways'] != 'UNKN']
    rwy_df = rwy_df.reset_index()
    
    # Read in Output_All_Points.csv from Lex
    output_all_points = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Output_All_Points.csv',encoding='cp1252')
    
    rwy_complete_df = pd.merge(rwy_df, output_all_points, on='REMARKS', how='left')
    rwy_complete_df = rwy_complete_df.drop_duplicates(subset = ['REMARKS'])
    std_rwyComplete = rwy_complete_df[['REMARKS','UAS_Location_Runways', 'UASLOCATION','REPORTINGFACILITY','RWYLOCATION']].reset_index()
    
    # Rename the 'UASLOCATION' field to 'IDENT' for merging purposes
    std_rwyComplete.rename(columns = {'REPORTINGFACILITY': 'IDENT'}, inplace = True)

    # Create List that contains each Standard Remark, and the Heading/Direction
    # information contained in each remark

    remark_uas_loc = []
    remarks = std_rwyComplete['REMARKS']

    # regular expression for any heading/direction
    headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?NM'

    # Loop through all remarks and search for the heading/direction regex above
    for i in range(len(remarks)):
        head_dir = re.findall(headir_regex, remarks[i])
        remark_uas_loc.append(remarks[i])
        remark_uas_loc.append(head_dir)

    # Split Remarks and Heading/Directions into two seperate lists and create
    # pandas dataframe
    remark = []
    uas_loc = []

    for i in range(0, len(remark_uas_loc), 2):
        remark.append(remark_uas_loc[i])
        uas_loc.append(remark_uas_loc[i+1])

    # Append UAS Distance
    std_rwyComplete['UAS_Distance'] = uas_loc
    
    # Export final DF to .csv 
    std_rwyComplete.to_csv('other2142_runways.csv', index = False)
    
    return std_rwyComplete

In [29]:
uas_rwy('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/unidentified_std_remarks.csv')

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance
0,0,Aircraft observed a UAS 100 feet above the Air...,[RWY30],NaN,NaN,NaN,[2 NM]
1,1,Aircraft observed an UAS at 200 feet above the...,[RWY8],NaN,NaN,NaN,[]
2,2,Aircraft observed a UAS off his right side at ...,[RWY23],NaN,NaN,NaN,[]
3,3,"Aircraft observed a red and white UAS at 1,000...","[RWY18L, RWY18C]",NaN,NaN,NaN,[]
4,4,"Aircraft observed a black UAS at 1,000 feet wh...",[RWY 32R],PAM,PAM,RWY 32R,[3 NM]
...,...,...,...,...,...,...,...
594,596,Aircraft reported a fixed wing UAS pass within...,[RWY 10],SAV,SAV,RWY 10,[1 NM]
595,597,Aircraft reported a grey quad-copter UAS off t...,[RWY 27L],SFB,SFB,RWY 27L,[]
596,598,Aircraft reported a black quad-copter UAS off ...,[RWY 3R],NaN,NaN,NaN,[]
597,599,Aircraft reported a NMAC with a UAS while on a...,[RWY05],NaN,NaN,NaN,[6 NM]
